## create data 

In [1]:
import pandas as pd

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    df = pd.read_csv("/kaggle/input/btcusdt-2023-6-9/btcusdt-2023-6_9.csv", index_col=0).reset_index(drop=True)
else:
    df = pd.read_csv("../lesson5-random-forests/btc-data/btcusdt-2023-6_9.csv", index_col=0).reset_index(drop=True)


print(df.shape)
df.head(3)

(11716, 6)


,time,open,high,low,close,vol
0,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675
1,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680
2,2023.06.01 00:30,27047.0,27077.4,27041.0,27054.9,275.080


In [2]:
# shift data 3 times, so that in one row there is information on the last 3 candles 
# therefore: the original candle data is the target data

df_s1 = df.shift(1).add_suffix("_s1")
df_s2 = df.shift(2).add_suffix("_s2")
df_s3 = df.shift(3).add_suffix("_s3")
df_s3 = df.shift(3).add_suffix("_s3")
df_s3 = df.shift(3).add_suffix("_s3")

print(df_s3.shape)
df_s3.head(3)

(11716, 6)


,time_s3,open_s3,high_s3,low_s3,close_s3,vol_s3
0,None,NaN,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,NaN


In [3]:
df_merge = pd.concat([df_s3,df_s2, df_s1, df], axis=1)
print(df_merge.shape)
df_merge.head(5)

(11716, 24)


,time_s3,open_s3,high_s3,low_s3,close_s3,vol_s3,time_s2,open_s2,high_s2,low_s2,...,high_s1,low_s1,close_s1,vol_s1,time,open,high,low,close,vol
0,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675
1,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,27108.1,27080.6,27096.9,386.675,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680
2,None,NaN,NaN,NaN,NaN,NaN,2023.06.01 00:00,27103.1,27108.1,27080.6,...,27096.9,27036.7,27047.0,408.680,2023.06.01 00:30,27047.0,27077.4,27041.0,27054.9,275.080
3,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675,2023.06.01 00:15,27096.9,27096.9,27036.7,...,27077.4,27041.0,27054.9,275.080,2023.06.01 00:45,27054.9,27084.0,27054.8,27084.0,218.143
4,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680,2023.06.01 00:30,27047.0,27077.4,27041.0,...,27084.0,27054.8,27084.0,218.143,2023.06.01 01:00,27084.0,27113.9,27073.5,27100.0,329.412


In [4]:
df_merge.columns

Index(['time_s3', 'open_s3', 'high_s3', 'low_s3', 'close_s3', 'vol_s3',
       'time_s2', 'open_s2', 'high_s2', 'low_s2', 'close_s2', 'vol_s2',
       'time_s1', 'open_s1', 'high_s1', 'low_s1', 'close_s1', 'vol_s1', 'time',
       'open', 'high', 'low', 'close', 'vol'],
      dtype='object')

In [8]:
df_merge.dropna().to_csv("nnbasic-btc-data.csv")

In [6]:
# do not use the time columns 

df_train = df_merge.filter(items = ['open_s3', 'high_s3', 'low_s3', 'close_s3', 'vol_s3',
       'open_s2', 'high_s2', 'low_s2', 'close_s2', 'vol_s2',
       'open_s1', 'high_s1', 'low_s1', 'close_s1', 'vol_s1', 
       'open', 'high', 'low', 'close', 'vol']).dropna()
print(df_train.shape)
df_train.head(3)

(11716, 20)


,open_s3,high_s3,low_s3,close_s3,vol_s3,open_s2,high_s2,low_s2,close_s2,vol_s2,open_s1,high_s1,low_s1,close_s1,vol_s1,open,high,low,close,vol
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27103.1,27108.1,27080.6,27096.9,386.675
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27103.1,27108.1,27080.6,27096.9,386.675,27096.9,27096.9,27036.7,27047.0,408.680
2,NaN,NaN,NaN,NaN,NaN,27103.1,27108.1,27080.6,27096.9,386.675,27096.9,27096.9,27036.7,27047.0,408.680,27047.0,27077.4,27041.0,27054.9,275.080
